In [1]:
using Plots, SparseArrays, DifferentialEquations, Arpack, LinearAlgebra

In [2]:
struct ReservoirComputer
    W_in
    W
    f
    gamma
    u
end

In [3]:
function evolve!(dr, r, rc::ReservoirComputer, t)
    dr[:] = (1-rc.gamma)*r + rc.gamma*(rc.f.(rc.W_in*vcat(1,rc.u(t)) + rc.W*r))
end

evolve! (generic function with 1 method)

In [4]:
ReservoirComputer(u, rho, sigma, gamma, f, Nu, Nr, bias_scale) = begin 
#     Compute W_in
    W_in = 2*rand(Nr, 1+Nu).-1
    W_in[:,1] *= bias_scale
    W_in[:,2:end] *= sigma
    
#     Compute W
    W = 2*sprand(Nr, Nr, 0.05).-1
    spectral_radius = abs(eigs(W, nev=1, ritzvec=false)[1][1])
    W *= rho/spectral_radius
    
    ReservoirComputer(W_in, W, f, gamma, u)
end

ReservoirComputer

In [5]:
include("../src/CaseStudies.jl")

Main.CaseStudies

In [6]:
lorenz = CaseStudies.get_lorenz(100);

In [7]:
rho = 0.8
sigma = 2.0
gamma = 0.25
f = tanh
Nu = 3
Nr = 10000
bias_scale = 1
rc = ReservoirComputer(lorenz, rho, sigma,gamma,f,Nu,Nr,bias_scale);

In [8]:
function burn_in(rc::ReservoirComputer, tspan)
    r0 = rand(size(rc.W)[1])
    prob = ODEProblem(evolve!, r0, tspan, rc)
    solve(prob).u[end]
end

burn_in (generic function with 1 method)

In [9]:
r = burn_in(rc, (0,20));

In [10]:
α = 0.1

0.1

In [11]:
function train(rc::ReservoirComputer, r0, α, tspan)
    prob = ODEProblem(evolve!, r0, tspan, rc)
    solution = solve(prob, dt=0.02);
    R = hcat(solution.u...)
    U = hcat(lorenz.(solution.t)...)
    ((R*R' - α*I) \ R*(U'))'
end

train (generic function with 1 method)

In [12]:
W_out = train(rc, r, α, (20.0,50.0));

In [13]:
struct TrainedResComp
    W_in
    W
    W_out
    f
    gamma
end

In [14]:
TrainedResComp(W_out, rc::ReservoirComputer) = TrainedResComp(
    rc.W_in, rc.W, W_out, rc.f, rc.gamma)

TrainedResComp

In [15]:
trc = TrainedResComp(W_out, rc);

In [16]:
function self_evolve!(dr, r, trc::TrainedResComp, t)
    dr[:] = (1-trc.gamma)*r + trc.gamma*(trc.f.(trc.W_in*vcat(1,trc.W_out*r) + trc.W*r))
end

self_evolve! (generic function with 1 method)

In [17]:
function predict(trc::TrainedResComp, r0, tspan)
    prob = ODEProblem(self_evolve!, r0, tspan, trc)
    solve(prob, dt=0.02);
end

predict (generic function with 1 method)

In [18]:
prediction = predict(trc, r, (50.0, 100.0));

In [19]:
system_prediction = [trc.W_out*v for v in prediction.u];

In [21]:
plot(lorenz, layout=3)